In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import torch
import unsloth
import pandas as pd
torch.cuda.is_available()

True

In [1]:
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel
from datasets import Dataset
from unsloth import is_bfloat16_supported

# Saving model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Warnings
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

c:\Users\rizky\Project\cybersec-llm\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\rizky\AppData\Local\Temp\ipykernel_11164\3850014559.py:3: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth.chat_templates import get_chat_template


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
max_seq_length = 5020
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="preemware/Prox-MistralHermes-7B",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)


==((====))==  Unsloth 2025.6.2: Fast Mistral patching. Transformers: 4.52.4.
   \\   /|    NVIDIA GeForce RTX 4060. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.7.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 3/3 [00:33<00:00, 11.23s/it]


In [4]:
data_prompt = """You are a cybersecurity expert. Your task is to answer the following multiple-choice questions by providing only the letter of the correct option.

## Input:
{}

## Output:
{}"""

In [5]:
text = """
Question: What is the most effective technique for identifying the behavior of potential polymorphic malware during malware analysis?
Options:

A. Passing strings fetched at runtime through a disassembler
B. Intercepting and modifying system calls with the ptrace system call
C. Examining the import address table for unexpected changes
D. Using sandboxing technologies to observe behavior

Correct Option: [D]

Explanation:

Using sandboxing technologies to observe behavior is the most effective technique for identifying the behavior of potential polymorphic malware. This approach allows analysts to see how malware operates in a controlled environment, making it possible to observe changes and behaviors without risking the integrity of the host system. Options such as 'Passing strings fetched at runtime through a disassembler' can be circumvented by malware that detects disassembler use. 'Intercepting and modifying system calls with the ptrace system call' might not reveal all behaviors, especially if the malware is aware and can disable or mislead ptrace. 'Examining the import address table for unexpected changes' is useful for static analysis but might miss runtime polymorphic behaviors where the malware modifies its code during execution.

---

Question: In the context of exploit development, what technique would most likely be used to carry out a privilege escalation attack on a modern Windows operating system?
Options:
A. Using a debugger to manipulate running processes and extract data
B. Creating a custom encryption algorithm for secure communications
C. Executing a buffer overflow attack to gain unauthorized access
D. Employing social engineering tactics to gain initial network access
"""

In [16]:
# Improved prompt format - cleaner and more direct
data_prompt = """You are a cybersecurity expert. Answer the following multiple-choice question by providing the letter of the correct option with format [choice] and give explanation.
Example:

Question: What is the most effective technique for identifying the behavior of potential polymorphic malware during malware analysis?
Options:
[
"SQL Injection via email input",
"Shell upload via form manipulation",
"Cross-site Scripting (XSS) using URL parameters",
"Denial of Service (DoS) through malformed packets"
]

Correct Option: [D]

Explanation:

Using sandboxing technologies to observe behavior is the most effective technique for identifying the behavior of potential polymorphic malware. This approach allows analysts to see how malware operates in a controlled environment, making it possible to observe changes and behaviors without risking the integrity of the host system. Options such as 'Passing strings fetched at runtime through a disassembler' can be circumvented by malware that detects disassembler use. 'Intercepting and modifying system calls with the ptrace system call' might not reveal all behaviors, especially if the malware is aware and can disable or mislead ptrace. 'Examining the import address table for unexpected changes' is useful for static analysis but might miss runtime polymorphic behaviors where the malware modifies its code during execution.

---

Question: {}

Options:
{}

Answer:"""

# Test question - separate the question from examples
test_question = "In exploit development, what technique is commonly applied to evade basic signature-based detection systems while maintaining the integrity of the exploit payload?"

test_options = """[
"XOR encoding in the payload",
"Multiplying payload size",
"Encrypting packet headers",
"Compressing the exploit code"
]"""

# Generate with more restrictive parameters
model = FastLanguageModel.for_inference(model)
inputs = tokenizer([
    data_prompt.format(test_question, test_options)
], return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs, 
    max_new_tokens=1024,  # Much smaller - just need 1-2 tokens for the answer
    temperature=0.3,   # Lower temperature for more deterministic output
    do_sample=True,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1  # Prevent repetition
)

answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
# Extract just the generated part
generated_text = answer[len(tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)):]
print(f"{generated_text.strip()}")

[A]

Explanation:

XOR encoding is a common technique used in exploit development to evade signature-based detection systems. It involves XOR-ing the payload with a key, which makes the payload appear random and unrecognizable to signature-based detection systems. The encoded payload still maintains its original functionality when decoded, ensuring the integrity of the exploit. Multiplying payload size would only increase the chances of triggering anomaly-based detection systems, encrypting packet headers is more relevant to network traffic analysis, and compressing the exploit code could potentially reduce its effectiveness due to the overhead of decompression.


In [17]:
del model, tokenizer

In [19]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()

9879